Some thoughts on doing satellite dodging

In [1]:
import numpy as np

In [2]:
# Let's calculate the typical efficiency of the telescope
# open shutter 1s, expose 14s, close shutter 1s, read 2s, open shutter 1s, 
# expose 14s, close shutter 1s, slew and settle 4.5s

30./(1+14+1+2+1+14+1+4.5)*100

77.92207792207793

In [3]:
# So, we are keeping the shutter open 78% of the time in standard observing
# (filter changes take ~2 min, sometimes we make a bigger slew, so it ends up around 72%)

In [7]:
# How much does a giant streak impact an observation?
total_pix = 3.02e9  # Total number of pixels
# we have 4kx4k CCDs, in 3x3 rafts. Full diameter is 5 rafts. 
scale = 0.2 # arcsec/pix
length = 4e3*3*5  # pix
width = 80  # pix
pix_hit = length*width

print('%i pixels hit, or %f%% of pixels' % (pix_hit, pix_hit/total_pix*100))

4800000 pixels hit, or 0.158940% of pixels


In [8]:
# If we take 1 extra second to dodge
30./(1+14+1+2+1+14+1+4.5+1)*100

75.9493670886076

In [9]:
# Taking one extra second to dodge, we lose 2% of our efficency, all to save 0.15% of 
# the pixels from being hit. 

How could we dodge satellites? 

The easiest way would be to forcast where there will be LOTS of satellites in the next hour, and try to avoid that part of the sky. This has the advantage of being fairly computationally cheap (just need to compute where we think thigs will be in the next ~hour). Doesn't need to be high precision. Easily fits in with out MDP system, we just make the forecast a basis function and give it a negative weight.

Next option:  Say we select a region of sky we want to observe. The next step is to tesselate it with pointings, sort the pointings using a traveling salesman solver, then copy the list if we want to take things in pairs. 

We could take our list of desired pointings, compute the expected execution times, calculate length of streaks. If there are lots of streaks, change the order of the pointings (maybe just which one we start on), and recompute. If we get to a list with no streaks, use it, otherwise, take the minimum.

This is going to be expensive (lots of satellite positions to calculate, lots of ra,dec to alt,az conversions). We probably need to insantiate the kinematic model inside the scheduler. Tough to say if we have enough precision for this to work.
